# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-5-nano'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:09<00:00, 37.87s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

'Title: Open-box Bobsweep Bio Self-Emptying Robot Vacuum for $227 + free shipping\nDetails: Use promo code "VIPSAVE10OFF" to get this price. It\'s the best deal we could find by $43. Buy Now at eBay\nFeatures: \nURL: https://www.dealnews.com/Open-box-Bobsweep-Bio-Self-Emptying-Robot-Vacuum-for-227-free-shipping/21811334.html?iref=rss-c196'

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Open-box Apple Watch SE 2 44mm Smartwatch for $121 + free shipping
Details: Use promo code "VIPSAVE10OFF" to get this price. It's the best price we've seen for this model. Buy Now at eBay
Features: 
URL: https://www.dealnews.com/Open-box-Apple-Watch-SE-2-44-mm-Smartwatch-for-121-free-shipping/21811350.html?iref=rss-c142

Title: Open-box Samsung U7900 Series 65" 4K U

In [10]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [11]:
result = get_recommendations()

In [12]:
len(result.deals)

5

In [13]:
result.deals[1]

Deal(product_description='The Toshiba M550 Series 55M550NU is a 55-inch QLED 4K UHD television with built-in Amazon Fire TV for streaming and smart home compatibility. It delivers sharp 4K picture with Dolby Vision, HDR10, HDR10+, and HLG support for vibrant color and contrast, enhancing movies and sports content. The set provides voice control and ecosystem support via Amazon Alexa and Apple HomeKit integration, making it easy to control with other smart devices. With four HDMI inputs, it accommodates multiple external devices, such as a Blu-ray player, game console, and streaming devices, while delivering a bright, appealing picture in living rooms.', price=200.0, url='https://www.dealnews.com/products/Toshiba/Toshiba-M550-Series-55-M550-NU-55-QLED-4-K-UHD-Fire-TV/489273.html?iref=rss-f1912')

In [14]:
from agents.scanner_agent import ScannerAgent

In [15]:
agent = ScannerAgent()
result = agent.scan()

In [16]:
result

DealSelection(deals=[Deal(product_description='Certified refurbished EcoFlow Delta Pro is a high-capacity portable power station designed for home backup and off-grid power needs. It features a 4096Wh lithium iron phosphate (LFP) battery that can deliver up to 4000W of AC output and supports 120/240V operation for a wide range of appliances. The battery chemistry promises long cycle life, and the unit can be expanded to larger capacities to reach up to 48kWh when additional modules are added. It is designed for quiet operation with an approximate 30 dB background and a portable, easy-to-set-up form factor. This power station is ideal for powering essential devices during outages, outdoor activities, or remote work setups.', price=1559.0, url='https://www.dealnews.com/Certified-Refurb-Eco-Flow-Delta-Pro-3-4-000-Wh-Power-Station-for-1-559-free-shipping/21811208.html?iref=rss-c142'), Deal(product_description='Samsung UN65U7900FFXZA is a 65-inch 4K UHD smart TV with a detailed 3840x2160 di